In [60]:
import sys
sys.path.insert(1, '/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/codes/')

In [61]:
from utilities import *

In [62]:
import warnings
warnings.filterwarnings('ignore')

In [63]:
df_master_index_all=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_all.pkl','rb'))

df_master_index_train=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_train.pkl','rb'))
df_master_index_val=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_val.pkl','rb'))
df_master_index_test=pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/split_index/df_master_index_test.pkl','rb'))

df_master_index_train_val=pd.concat([df_master_index_train,df_master_index_val])

In [64]:
df_feats_clinical = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_clinical.pkl','rb'))
df_feats_icd_transform = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_icd_transform.pkl','rb'))

df_icu_graph_features = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_icu_graph_features.pkl','rb'))

df_feats_ppg_train_all = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_train_separate.pkl','rb'))
df_feats_ppg_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_val_separate.pkl','rb'))
df_feats_ppg_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/df_feats_ppg_test.pkl','rb'))


feats_n2v_all_train = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_train.pkl','rb'))
feats_n2v_all_val = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_val.pkl','rb'))
feats_n2v_all_test = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/feats_n2v_all_test.pkl','rb'))



Outcome_labels = pickle.load(open('/drives/drive1/home/changyu_yin/Desktop/PSG/PSG_AMIA_NEJM/feature/Outcome_labels.pkl','rb'))

In [65]:
df_feats_clinical_icd = pd.merge(df_feats_clinical,
                                 df_feats_icd_transform,
                                 on='ENCRYPTED_HOSP_ENCOUNTER',
                                 how='left')

In [66]:
cutoff_y = np.quantile(Outcome_labels['LOS_postoperative'],0.75)
cutoff_y
Outcome_labels['PLOS'] =  (Outcome_labels['LOS_postoperative']>cutoff_y)*1
del Outcome_labels['LOS_postoperative']

Outcome_labels_train = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_val = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

Outcome_labels_test = Outcome_labels[Outcome_labels['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)[['ENCRYPTED_HOSP_ENCOUNTER','PLOS']]

len(Outcome_labels_train)+len(Outcome_labels_val)+len(Outcome_labels_test)-len(Outcome_labels)

0

In [67]:
df_icu_graph_features_train = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_val = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

df_icu_graph_features_test = df_icu_graph_features[df_icu_graph_features['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True)

len(df_icu_graph_features_train)+len(df_icu_graph_features_val)+len(df_icu_graph_features_test)-len(df_icu_graph_features)

0

In [68]:
df_feats_clinical_icd_ppg_icug_train = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_train['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_train_all,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_train,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_train,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_val = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_val['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_val,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_val,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


df_feats_clinical_icd_ppg_icug_test = df_feats_clinical_icd[df_feats_clinical_icd['ENCRYPTED_HOSP_ENCOUNTER'].\
                                      isin(df_master_index_test['ENCRYPTED_HOSP_ENCOUNTER'])].\
                        reset_index(drop=True).\
                        merge(df_feats_ppg_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
                        merge(df_icu_graph_features_test,how='left',on='ENCRYPTED_HOSP_ENCOUNTER').\
            merge(feats_n2v_all_test,how='left',
                   on='ENCRYPTED_HOSP_ENCOUNTER')


len(df_feats_clinical_icd_ppg_icug_train)+len(df_feats_clinical_icd_ppg_icug_val)+len(df_feats_clinical_icd_ppg_icug_test)-len(df_feats_clinical_icd)

0

In [103]:
y_train = Outcome_labels_train
y_val = Outcome_labels_val
y_test = Outcome_labels_test

X_train = y_train[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_train,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)
X_val = y_val[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_val,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_test = y_test[['ENCRYPTED_HOSP_ENCOUNTER']].\
            merge(df_feats_clinical_icd_ppg_icug_test,
                  how='left',
                  on='ENCRYPTED_HOSP_ENCOUNTER').\
            reset_index(drop=True)

X_train_val = pd.concat([X_train,X_val]).reset_index(drop=True)
y_train_val = pd.concat([y_train,y_val]).reset_index(drop=True)

In [104]:
X_train.shape,X_val.shape,X_train_val.shape,X_test.shape,y_train.shape,y_val.shape,y_train_val.shape,y_test.shape

((109318, 4503),
 (13665, 4503),
 (122983, 4503),
 (13664, 4503),
 (109318, 2),
 (13665, 2),
 (122983, 2),
 (13664, 2))

In [105]:
X_train_val.columns.value_counts()

ENCRYPTED_HOSP_ENCOUNTER    1
icd10_transformed_C14       1
icd10_transformed_C20       1
icd10_transformed_C19       1
icd10_transformed_C18       1
                           ..
CPT 1_35560                 1
CPT 1_35563                 1
CPT 1_35565                 1
CPT 1_35566                 1
Node2Vec_127                1
Length: 4503, dtype: int64

# Tuning

In [106]:
del sys.modules["utilities"]
from utilities import *

In [107]:
np.random.seed(666)

In [108]:
a = np.zeros(len(X_train))-1 
b = np.ones(len(X_val)) 
pds = PredefinedSplit(np.concatenate([a,b]))

In [112]:
pipe = Pipeline(steps=[
                       ("imputer", SimpleImputer(missing_values=np.nan, strategy='mean')),
                       ("scaler", MinMaxScaler()),
                       ("classifier", XGBClassifier(n_jobs=-1)) 
                        ])

In [114]:
clf = BayesSearchCV(estimator = pipe,
                   search_spaces={
                    'classifier__n_estimators':Integer(50,600),
                    'classifier__learning_rate': Real(0.01,0.3,'log-uniform'),
                    'classifier__max_depth': Integer(3,15),
                    'classifier__subsample': Real(0.6,1,'uniform'), 
                    'classifier__colsample_bytree': Real(0.5,1,'uniform')
                    # 'gamma': Real(1e-6,10,'log-uniform'), 
                    #'eta': Real(0.01,0.5,'log-uniform'),
                    #'min_child_weight':Integer(1,20),
                   },
                   n_iter = 10,
                   cv=pds,
                   scoring = 'roc_auc',
                   n_jobs=32,
                   refit=True,
                   verbose=4,
                   random_state=666
                   ) 

In [115]:
clf.fit(X_train_val.iloc[:,1:],y_train_val.iloc[:,1:].values.ravel())

Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
Fitting 1 folds for each of 1 candidates, totalling 1 fits
[CV 1/1] END classifier__colsample_bytree=0.7965961755821049, classifier__learning_rate=0.019758622345048785, classifier__max_depth=12, classifier__n_estimators=417, classifier__subsample=0.773808175844738;, score=0.868 total time=196.4min
[CV 1/1] END classifier__colsample_bytree=0.9401410615897656, classifier__learning_rate=0.015613685054942743, classifier__max_depth=5, classifier__n_estimators=56, classi

BayesSearchCV(cv=PredefinedSplit(test_fold=array([-1, -1, ...,  1,  1])),
              estimator=Pipeline(steps=[('imputer', SimpleImputer()),
                                        ('scaler', MinMaxScaler()),
                                        ('classifier',
                                         XGBClassifier(base_score=None,
                                                       booster=None,
                                                       callbacks=None,
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       early_stopping_rounds=None,
                                                       enable_categorical=False,
                                                       eval_metric=...
                             'classifier__learning_rate': Real(low=0.01, high=0.3, prior='log-uniform', transform='normalize'),
                             'classifier__max_depth': Integer(low=3, high=15, prior='uniform', transform='normalize'),
                             'classifier__n_estimators': Integer(low=50, high=600, prior='uniform', transform='normalize'),
                             'classifier__subsample': Real(low=0.6, high=1, prior='uniform', transform='normalize')},
              verbose=4)

In [ ]:
print(clf.best_params_)
best_m = clf.best_estimator_
pred = best_m.predict(X_test.iloc[:,1:])
pred_prob = best_m.predict_proba(X_test.iloc[:,1:])[:,1]
print(min(pred_prob),max(pred_prob))

print('Best Validation AUROC: {}'.format(clf.best_score_))

print('Test AUROC: {}'.format(roc_auc_score(y_test.iloc[:,1:], pred_prob)))
print('Test F1-score: {}'.format(f1_score(y_test.iloc[:,1:], pred)))
print('Test Recall: {}'.format(recall_score(y_test.iloc[:,1:], pred)))
print('Test Precision: {}'.format(precision_score(y_test.iloc[:,1:], pred)))
print('Test Accuracy: {}'.format(accuracy_score(y_test.iloc[:,1:], pred)))

In [13]:
#experiment_baseline(X_train, X_val, y_train, y_val, seeds=[2023])

In [26]:
#experiment_baseline(X_train_val, X_test, y_train_val, y_test, seeds=[2023])